In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1337)

eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 50 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4

In [2]:
class scaledDotProduct(nn.Module):
    '''
        Attention(Q, K, V ) = softmax( QK^T/√d_k)V 
    
    '''
    #Takes number of embedded, head_size, context length
    def __init__(self, embn, hdim, con_l,  drop=0.0):

        super(scaledDotProduct, self).__init__()
        #dim is (d_k) when sqrt'd it is meant to counter small gradients in large sets of queries and keys
        self.k = nn.Linear(embn, hdim, bias=False)
        self.q = nn.Linear(embn, hdim, bias=False)
        self.v = nn.Linear(embn, hdim, bias=False)
        self.d_k = np.sqrt(hdim)
        self.hdim = hdim
        self.block_size = con_l
        

        self.k_cache = torch.zeros((batch_size, self.block_size,self.hdim))
        self.v_cache = torch.zeros((batch_size, self.block_size,self.hdim))
        self.cache_index = 0

        self.register_buffer('mask', torch.tril(torch.ones(con_l,con_l)))
        #Simple drop out 
        self.drop = nn.Dropout(drop)

    def forward(self, x, ret_att=False):
        #batch X length X dim
        B,T,C = x.shape
        k = self.k(x)
        q = self.q(x)
        v = self.v(x)

        #if self.k_cache is None or self.v_cache is None:
        #    self.k_cache = torch.zeros(B,self.block_size, self.hdim, device=x.device)
        #    self.v_cache = torch.zeros(B,self.block_size, self.hdim, device=x.device)
        #    self.cache_index = 0
        self.k_cache = self.k_cache.to(x.device)
        self.v_cache = self.v_cache.to(x.device)
        if self.cache_index + T <= self.block_size:
            self.k_cache[:B, self.cache_index:self.cache_index + T, :] = k
            self.v_cache[:B, self.cache_index:self.cache_index + T, :] = v
        else:
            # shift the tokens one step back
            shift = self.cache_index + T - block_size # Shift will always be 1
            self.k_cache[:B, :-shift, :] = self.k_cache[:, shift:, :].clone()
            self.v_cache[:B, :-shift, :] = self.v_cache[:, shift:, :].clone()
            self.k_cache[:B, -T:, :] = k
            self.v_cache[:B, -T:, :] = v

        self.cache_index = min(self.cache_index + T, self.block_size)

        
        n = torch.matmul(q, self.k_cache[:B,:,:].transpose(-2,-1)) * k.shape[-1]**-0.5 #BxTxhdim
        n = n.masked_fill(self.mask[:T,:T]==0, float('-inf'))
        #Drop out referenced later in paper but not in original diagram
        att = self.drop(F.softmax(n, dim=-1))

        

        out = torch.matmul(att, self.v_cache[:B,:,:])
        if ret_att:
            return out, att 
        return out
        
        


In [3]:
attention_test = scaledDotProduct(384,10,100)
#
v2 = torch.rand(2,100,384)

attention_test(v2)

tensor([[[ 7.3799e-02, -2.1080e-01, -1.6968e-01,  ...,  1.2158e-01,
           1.4039e-01, -7.1718e-02],
         [-4.0966e-02, -2.2383e-01, -2.5205e-02,  ...,  1.3851e-01,
           1.0924e-01,  2.6803e-02],
         [ 3.7568e-05, -1.2587e-01,  4.2005e-02,  ...,  8.6697e-02,
           1.0704e-01,  3.7151e-02],
         ...,
         [-4.2986e-02, -5.4267e-02,  5.2033e-02,  ..., -2.0862e-02,
           1.0813e-01,  9.0752e-02],
         [-4.1764e-02, -5.5534e-02,  5.1420e-02,  ..., -2.3993e-02,
           1.0773e-01,  9.0928e-02],
         [-4.1788e-02, -5.7114e-02,  5.0740e-02,  ..., -1.7963e-02,
           1.0715e-01,  8.8348e-02]],

        [[ 1.4407e-01, -1.0904e-01,  4.4700e-01,  ...,  8.7784e-02,
          -7.3098e-02,  2.7388e-02],
         [ 2.1906e-01,  4.2942e-04,  5.2665e-01,  ...,  7.0088e-02,
          -7.9570e-02,  2.4258e-01],
         [ 1.3485e-01, -1.5217e-02,  3.5493e-01,  ...,  5.0866e-02,
          -1.5452e-02,  1.7811e-01],
         ...,
         [-3.9585e-02, -5

In [4]:
class multiHeadedAttention(nn.Module):
    def __init__(self, n_heads, dims, embn, con_l, dropout=0.0):
        super(multiHeadedAttention, self).__init__()
        #d_k=d_v = dims/h

        self.n_heads = n_heads

        self.attn = nn.ModuleList([scaledDotProduct(embn, dims, con_l) for _ in range(n_heads)])
        #Final linear layer after concat and attention
        self.fc = nn.Linear(n_heads*dims, embn)

        self.drop = nn.Dropout(dropout)
        

    def forward(self, x):
        out = torch.cat([h(x) for h in self.attn], dim=-1)
        out = self.drop(self.fc(out))
        return out

In [5]:
#heads, d_model, d_km d_v as per the paper
torch.manual_seed(1337)
multiHead = multiHeadedAttention(6, 50, 384, 512, dropout=0.2)

#batches, dims, dimensionalityxn_heads

v = torch.rand(1,512,384)


print(multiHead(v))

tensor([[[-0.1094,  0.3953, -0.0000,  ..., -0.0000,  0.2078, -0.0762],
         [-0.0667,  0.3789, -0.0155,  ..., -0.2896,  0.0174,  0.0112],
         [-0.0823,  0.3302,  0.0112,  ..., -0.2928, -0.0329,  0.0370],
         ...,
         [-0.0032,  0.2465,  0.0285,  ..., -0.2252, -0.0884,  0.0711],
         [-0.0000,  0.2461,  0.0000,  ..., -0.0000, -0.0877,  0.0000],
         [-0.0000,  0.2464,  0.0287,  ..., -0.2257, -0.0000,  0.0698]]],
       grad_fn=<MulBackward0>)


In [6]:
class positionFeedFoward(nn.Module):
    def __init__(self, inp, hid, drop=0.0):
        super(positionFeedFoward, self).__init__()
        self.w1 = nn.Linear(inp,4*hid)
        self.w2 = nn.Linear(4*hid,inp)
        self.drop = nn.Dropout(drop)

    def forward(self, x):

        x = self.w2(F.relu(self.w1(x)))
        x = self.drop(x)

        return x

In [7]:
class Decoder(nn.Module):
    '''Combinds MultiHeadedAttention and FeeForward, three layers'''
    def __init__(self, nheads, embn, con_l,  drop=0.0):
        super(Decoder, self).__init__()
        head_size = embn // nheads
        self.slf_attn = multiHeadedAttention(nheads, head_size,embn, con_l,  dropout=drop)
        
        self.ffn = positionFeedFoward(embn, embn, drop=drop)

        self.norm1 = nn.LayerNorm(embn)
        self.norm2 = nn.LayerNorm(embn)

    def forward(self, x):
        tet = self.slf_attn(self.norm1(x))
        x = x + self.slf_attn(self.norm1(x))
        x = x + self.ffn(self.norm2(x))

        return x

In [8]:

#heads, d_model, d_km d_v as per the paper
torch.manual_seed(1337)
multiHead = multiHeadedAttention(6, 50, 384, 512, dropout=0.2)

#batches, dims, dimensionalityxn_heads

v = torch.rand(1,512,384)


print(multiHead(v))

tensor([[[-0.1094,  0.3953, -0.0000,  ..., -0.0000,  0.2078, -0.0762],
         [-0.0667,  0.3789, -0.0155,  ..., -0.2896,  0.0174,  0.0112],
         [-0.0823,  0.3302,  0.0112,  ..., -0.2928, -0.0329,  0.0370],
         ...,
         [-0.0032,  0.2465,  0.0285,  ..., -0.2252, -0.0884,  0.0711],
         [-0.0000,  0.2461,  0.0000,  ..., -0.0000, -0.0877,  0.0000],
         [-0.0000,  0.2464,  0.0287,  ..., -0.2257, -0.0000,  0.0698]]],
       grad_fn=<MulBackward0>)


In [9]:
#Pytoch version adapted from here https://pub.aimind.so/creating-sinusoidal-positional-embedding-from-scratch-in-pytorch-98c49e153d6

class PosEncoding(nn.Module):
    def __init__(self, hid, n_pos=200):
        super(PosEncoding, self).__init__()

        self.register_buffer('pos_table', self._get_sinusoid_encoding_table(n_pos, hid))

    def _get_sinusoid_encoding_table(self, n_pos, hid):

        if hid %2 != 0:
            raise ValueError("Sinusoidal positional embedding cannot apply to odd token embedding dim={}".format(hid))
        
        positions = torch.arange(0,n_pos).unsqueeze_(1)
        embeds = torch.zeros(n_pos, hid)

        denom = torch.pow(10000, 2 * torch.arange(0, hid//2)/2)
        embeds[:, 0::2] = torch.sin(positions/denom)
        embeds[:, 1::2] = torch.cos(positions/denom)
        embeds = embeds.unsqueeze(0)

        return embeds
    
    def forward(self, x):
        return x + self.pos_table[:, :x.size(1)].clone().detach()

In [10]:
class languageModel(nn.Module):
    '''Decoder model'''
    def __init__(
            self, n_vocab, embn, n_layers, n_head,  dropout=0.2 , con_l=200
    ):
        super(languageModel, self).__init__()
        self.con_l = con_l
        self.word_emb = nn.Embedding(n_vocab, embn)
        self.pos_enc = nn.Embedding(con_l, embn)
        self.stack = nn.Sequential(
            *[Decoder( n_head, embn, con_l,  drop=dropout) for _ in range(n_layers)]
        )
       
        self.layer_norm = nn.LayerNorm(embn)
        self.fc = nn.Linear(embn, n_vocab)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x, tar=None):
        #batch, time
        B, T = x.shape

        tok = self.word_emb(x)
        pos = self.pos_enc(torch.arange(T, device=device))
        x = tok + pos
        x = self.stack(x)
        x = self.layer_norm(x)
        logits = self.fc(x)

        if tar is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            tar = tar.view(B*T)
            loss = F.cross_entropy(logits, tar)

        return logits, loss
    
    def generate(self, x, max_length):
        #x is a BxT array of in current context
        for _ in range(max_length):
            x_cond = x[:, -self.con_l:]
            logits, loss = self(x_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            x_next = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, x_next), dim=1)

        return x

In [11]:
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

torch.manual_seed(1337)

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits = model(X)[0]
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = Y.view(B*T)
            loss = F.cross_entropy(logits, targets)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [12]:
print(vocab_size)

65


In [13]:
model = languageModel(vocab_size,  384,6, 6,  con_l=block_size
    )
m = model.to(device)
# print the number of parameters in the model
#print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
print(next(m.parameters()).is_cuda)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    #B, T, C = logits.shape
    #logits = logits.view(B*T, C)
    #targets = yb.view(B*T)
    #loss = F.cross_entropy(logits, targets)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_length=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

True
step 0: train loss 4.2128, val loss 4.2117


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
model = languageModel(vocab_size,  384,6, 6,  con_l=block_size
    ).to(device)
model.load_state_dict(torch.load("shake_spear.pth", weights_only=False))

No fused RMSNorm


AttributeError: Can't get attribute 'args' on <module '__main__'>

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(count_parameters(m))

In [ ]:
print(decode(context[0].tolist()))
print("===========================")
print(decode(m.generate(context, max_length=500)[0].tolist()))

In [ ]:
sinp = "wherefore art thou"
sinp = torch.tensor(encode(sinp)).unsqueeze(0).to(device)
print(decode(m.generate(sinp, max_length=500)[0].tolist()))